- pokemon_data.csv - 포켓몬의 다양한 정보를 가진 원본 csv 파일
- description.csv - GPT를 이용해 포켓몬 당 설명 3개 저장한 csv 파일

1. pokemon_data.csv의 name 열은 lower case로 저장되어 있음.
2. description.csv와 가끔 명칭이 다른 포켓몬이 있음.

위 두가지를 전처리

In [ ]:
import pandas as pd

df_full = pd.read_csv('/mnt/nas-drive/pys/cuketmon/datasets/pokemon_data.csv')
df_desc = pd.read_csv('description.csv')

df_desc['name'] = df_desc['name'].astype(str).str.lower()

df_desc.to_csv('description_lowercase.csv', index=False)

print("✅ name 컬럼을 소문자로 변환하여 description_lowercase.csv로 저장 완료!")

# name 컬럼만 추출 후 공백 제거
names_full = set(df_full['name'].astype(str).str.strip())
names_desc = set(df_desc['name'].astype(str).str.strip())

# 차이 계산
missing_in_desc = names_full - names_desc

# 결과 출력
print(f"✅ description.csv에 없는 name 수: {len(missing_in_desc)}")
for name in sorted(missing_in_desc):
    print(f"{name}")

✅ name 컬럼을 소문자로 변환하여 description_lowercase.csv로 저장 완료!
✅ description.csv에 없는 name 수: 0


descrition.csv는 name에 3개의 열이 중복됨.
name을 하나로 만들고 description 열을 3개 만들어서 전처리

In [ ]:
import pandas as pd

# description.csv 불러오기
df_desc = pd.read_csv('description_lowercase.csv')
df_desc['name'] = df_desc['name'].str.lower()

# 각 name마다 description을 리스트로 묶음
grouped = df_desc.groupby('name')['description'].apply(list)

# 리스트를 개별 열로 나눔
max_desc = 3  # 최대 설명 개수
df_wide = grouped.apply(lambda x: x[:max_desc] + [""] * (max_desc - len(x))).apply(pd.Series)
df_wide.columns = [f'description{i+1}' for i in range(max_desc)]
df_wide = df_wide.reset_index()

# 저장
df_wide.to_csv('description_wide.csv', index=False)
print("✅ description_wide.csv 생성 완료 (description1~3 열 포함)")

✅ description_wide.csv 생성 완료 (description1~3 열 포함)


설명을 랜덤으로 pokemon_data.csv에 매핑

In [ ]:
# 1. Load 데이터
df_pokemon = pd.read_csv('/mnt/nas-drive/pys/cuketmon/datasets/pokemon_data.csv')
df_desc = pd.read_csv('description_wide.csv')  # name, description1~3

# 2. description1~3 중 랜덤으로 고를 준비
# name → [description1, description2, description3] 구조로 딕셔너리 만들기
desc_dict = df_desc.set_index('name')[['description1', 'description2', 'description3']].to_dict(orient='index')

# 3. pokemon 데이터에 각 행마다 랜덤 설명 넣기
def get_random_description(name):
    entry = desc_dict.get(name)
    if entry:
        options = [entry['description1'], entry['description2'], entry['description3']]
        options = [desc for desc in options if pd.notna(desc) and desc != ""]
        return random.choice(options) if options else ""
    return ""

df_pokemon['description'] = df_pokemon['name'].apply(get_random_description)

# 4. 저장
df_pokemon.to_csv('pokemon_data_with_random_description.csv', index=False)
print("✅ 각 행마다 랜덤 description 추가 완료!")

✅ 각 행마다 랜덤 description 추가 완료!
